https://github.com/distil/ams-tools

In [ ]:
from dp import presto_tools
from prestodb.exceptions import PrestoQueryError
from time import gmtime, strftime, time
from slacker import Slacker
from os.path import expanduser, realpath, dirname
import pygsheets
import pandas as pd

In [ ]:
it = presto_tools.PrestoConnector()
it.get_presto_conn()

In [ ]:
start_time = time()

# set the directory vars
home = dirname(realpath(__file__))
sqldir = home + "/amssql/"
outdir = home + "/outputs/"
prvdir = home + "/private/"

In [ ]:
# timestamp
hour = strftime("%Y-%m-%d_%H-%M", gmtime())
url_base = "https://docs.google.com/spreadsheets/d/"

with open(prvdir+'apikey', 'r') as f:
    apikey = f.readline().strip()
    
slack = Slacker(apikey)

In [ ]:
# create the daily report and save to the outputs dir
def gauth():
    gc = pygsheets.authorize(service_file=prvdir + 'dp-to-pygsheets.json')
    return gc

In [ ]:
# Get the google sheet
def getqueries():
    sh = gc.open_by_key('1UUp2TOeC4i2DaNT1FEzM6C-QfwHvP_32gJDisLvC0Ak')
    wks = sh.worksheet_by_title('queries')
    queries = wks.get_all_records()
    gsheeturl = sh.url
    return {'queries': queries, 'gsheeturl': gsheeturl}

In [ ]:
# Helper function to check if the multiplier is actually a number
def multchecker(multiplier):
    multiplier = multiplier if isinstance(multiplier, (int, float)) else 1
    return multiplier

In [ ]:
# What does this function do???
def applyformat(gsheetid):
    return {"autoResizeDimensions":
            {"dimensions":
                {"sheetId": gsheetid,
                    "dimension": "COLUMNS",
                    "startIndex": 0}}}

In [ ]:
def sendtoslack(attachments, channel):
    try:
        slack.chat.post_message(
            channel,
            'ALERT',
            attachments=attachments,
            as_user='true'
        )
    except Exception as e:
        attachment = {}
        attachment['title'] = 'One of your jobs is sending to a bad channel'
        attachment['text'] = 'Either I\'m not invited, or it don\'t exist'
        attachments = [attachment]
        slack.chat.post_message(
            deflogschan,
            'BRUH',
            attachments=attachments,
            as_user='true'
        )

In [ ]:
def sendtogsheet(gsheetdict, channel):
    data = gsheetdict.get('data')
    aq = gsheetdict.get('aq')
    aratio = gsheetdict.get('aratio')
    sh = gc.create(
        hour + ' ' + qname,
        parent_id='1t9KzV_lOFRpki56WNYRER7MQsdyYT_LW'
    )
    wks = sh.sheet1
    wks.title = 'attack info'
    wks2 = sh.add_worksheet('query detail')
    # set the values of a pandas dataframe to sheet
    data['start'] = pd.to_datetime(data['start']).dt.strftime('%Y/%m/%d %H:%M:%S')
    data['max_time'] = pd.to_datetime(data['max_time']).dt.strftime('%Y/%m/%d %H:%M:%S')
    wks.set_dataframe(data, (1, 1))
    gc.sh_batch_update(sh.id, applyformat(wks.id))
    wks2.update_cell('A1', aq)
    url = url_base + str(sh.id) + "/edit#gid=" + str(wks.id)

    # post current days file csm-team slack
    attachment = {}
    attachment['title'] = 'Attack in Progress, ' + qname
    attachment['title_link'] = url
    attachment['text'] = 'Multiplier: ' + str(round(aratio, 1)) + 'x'
    attachments = [attachment]
    sendtoslack(attachments, channel)

In [ ]:
def tcheck(data, multiplier, qname):
    try:
        aratio = data.requests/data.threshold
        if (aratio >= multiplier).any(): # the .any() here seems unnecessary
            aratio = float(aratio)
            # print(str(aratio) + ' ' + qname)
            alertdict = {
                'data': data,
                'aq': aq,
                'aratio': aratio
            }
            return(alertdict)
        else:
            pass
    except Exception as e:
        attachment = {}
        attachment['title'] = 'Could not determine requests/threshold'
        attachment['text'] = 'Likely requests count was 0 for ' + qname
        attachments = [attachment]
        sendtoslack(attachments, channel)

In [ ]:
def dpquery(q):
    try:
        data = it.query(q) # it = presto_tools.PrestoConnector()
        return(data)
    except PrestoQueryError as e:
        attachment = {}
        attachment['title'] = 'Query Failed on ' + qname
        attachment['title_link'] = gsheeturl
        attachment['text'] = e
        attachments = [attachment]
        sendtoslack(attachments, 'attackmon-logs')
    except TypeError as e:
        attachment = {}
        attachment['title'] = 'Query failed ' + qname
        attachment['title_link'] = gsheeturl
        attachment['text'] = 'Ratio requests/thresholds not determined'
        attachments = [attachment]
        sendtoslack(attachments, 'attackmon-logs')

In [ ]:
# Monitor section
# there are only 2 types of queries, depending if API or WEB
# The function below populates the query with the values from gsheet

def baq(q): # for q in queries
    ac = q.get('and_condition', '')
    ac = ac if '\'' not in ac else ac.replace('\'', '\"')
    if q.get('platform') == 'api':
        with open(sqldir + 'apimonitor.sql', 'r') as f:
            aq = f.read().format(
                domain=q.get('domain'),
                platform=q.get('platform'),
                threshold_one=q.get('threshold_one'),
                threshold_two=q.get('threshold_two'),
                threshold_three=q.get('threshold_three'),
                account_id=q.get('account_id'),
                domain_id=q.get('domain_id'),
                ilike_url=q.get('ilike_url'),
                and_condition=q.get('and_condition', ''),
                note=q.get('note'),
                multiplier=q.get('multiplier', 1),
                ac=ac
            )
            return(aq)
    elif q.get('platform') == 'web':
        with open(sqldir + 'webmonitor.sql', 'r') as f:
            aq = f.read().format(
                domain=q.get('domain'),
                platform=q.get('platform'),
                threshold_one=q.get('threshold_one'),
                threshold_two=q.get('threshold_two'),
                threshold_three=q.get('threshold_three'),
                account_id=q.get('account_id'),
                domain_id=q.get('domain_id'),
                ilike_url=q.get('ilike_url'),
                and_condition=q.get('and_condition', ''),
                note=q.get('note'),
                multiplier=q.get('multiplier', 1),
                ac=ac
            )
            return(aq)

In [ ]:
# Median section, same as aboce
def gmq(q):
    if q.get('platform') == 'api':
        with open(sqldir + 'apimedian.sql', 'r') as f:
            mquery = f.read().format(
                domain=q.get('domain'),
                platform=q.get('platform'),
                account_id=q.get('account_id'),
                domain_id=q.get('domain_id'),
                ilike_url=q.get('ilike_url'),
                and_condition=q.get('and_condition', ''),
            )
            return(mquery)
    elif q.get('platform') == 'web':
        with open(sqldir + 'webmedian.sql', 'r') as f:
            mquery = f.read().format(
                domain=q.get('domain'),
                platform=q.get('platform'),
                account_id=q.get('account_id'),
                domain_id=q.get('domain_id'),
                ilike_url=q.get('ilike_url'),
                and_condition=q.get('and_condition', ''),
            )
            return(mquery)
    else:
        attachment = {}
        attachment['title'] = 'Must specify web or api for ' + qname
        attachment['title_link'] = gsheeturl
        attachment['text'] = 'Check your query...'
        attachments = [attachment]
        sendtoslack(attachments, 'attackmon-logs')

In [ ]:
# Main

gc = gauth()
querydict = getqueries()
queries = querydict.get('queries')
gsheeturl = querydict.get('gsheeturl')
defchan = 'attack-ams-presto'
deflogschan = 'attackmon-logs'
jobs = 0

for q in queries:
    if q.get('status') == 'pause':
        pass
    elif q['name'] and q['ilike_url'] and q['account_id'] and \
            q['domain_id']:
        note = q.get('note')
        qname = q.get('name')
        and_condition = q.get('and_condition')
        if q.get('note'):
            note = note if '\'' not in note else 'NO QUOTES IN NOTE PLEASE'
            q.update({'note': note})
        if q.get('and_condition'):
            ac = and_condition.count('\'') % 2
            and_condition = and_condition if ac == 0 else ''
            q.update({'and_condition': and_condition})
        channel = q.get('channel') if q.get('channel') else defchan
        # mq = gmq(q)
        # thresholds = dpquery(mq)
        # thresholds = thresholds.to_dict('records')
        # thresholds = thresholds[0]
        # q.update(thresholds)
        # break
        aq = baq(q)
        # print(aq)
        data = dpquery(aq)
        multiplier = multchecker(q.get('multiplier', 1))
        alertdict = tcheck(data, multiplier, qname)
        if alertdict:
            sendtogsheet(alertdict, channel)
        else:
            pass
        jobs += 1
    else:
        attachment = {}
        attachment['title'] = 'Some params missing from ' + qname
        attachment['title_link'] = gsheeturl
        attachment['text'] = 'Check your query...'
        attachments = [attachment]
        sendtoslack(attachments, deflogschan)

In [ ]:
# ending
finishtime = ("--- %s seconds ---" % round((time() - start_time), 3))
attachment = {}
attachment['title'] = str(jobs) + ' Jobs Completed'
attachment['text'] = finishtime + ' ' + hour
attachments = [attachment]
sendtoslack(attachments, deflogschan)